# 🎬 Film Öneri Sistemi - Veri Bilimi Uygulaması

Bu notebook, film verilerini kullanarak içerik tabanlı öneri sistemi geliştirme sürecini adım adım göstermektedir.

## 📋 İçerik
1. **Veri Yükleme ve Keşif**
2. **Veri Ön İşleme**
3. **Keşifsel Veri Analizi (EDA)**
4. **Korelasyon Analizi**
5. **Film Öneri Sistemi**
6. **Regresyon Analizi**
7. **Sonuçlar ve Değerlendirme**

## 1. 📊 Gerekli Kütüphanelerin Yüklenmesi

In [ ]:
# Gerekli kütüphaneleri import et
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Grafik ayarları
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("✅ Tüm kütüphaneler başarıyla yüklendi!")

## 2. 📁 Veri Yükleme ve İlk İnceleme

In [ ]:
# Veri setini yükle
df = pd.read_excel('../data/filmtv_movies.xlsx')

print(f"📊 Veri seti boyutu: {df.shape}")
print(f"🎬 Toplam film sayısı: {len(df)}")
print(f"📋 Özellik sayısı: {df.shape[1]}")

# İlk 5 kaydı göster
print("\n🎯 İlk 5 film:")
df.head()

In [ ]:
# Veri seti hakkında genel bilgi
print("📋 Veri Seti Bilgileri:")
print(df.info())

print("\n📊 Sayısal Sütunların İstatistikleri:")
df.describe()

In [ ]:
# Eksik değerleri kontrol et
print("🔍 Eksik Değer Kontrolü:")
missing_values = df.isnull().sum()
if missing_values.sum() == 0:
    print("✅ Veri setinde eksik değer bulunmuyor!")
else:
    print(missing_values[missing_values > 0])

# Benzersiz değerleri kontrol et
print("\n🎭 Kategorik Sütunlardaki Benzersiz Değerler:")
categorical_cols = ['genre', 'country', 'directors']
for col in categorical_cols:
    print(f"{col}: {df[col].nunique()} benzersiz değer")
    print(f"  Örnekler: {df[col].unique()[:5]}\n")

## 3. 🔧 Veri Ön İşleme

In [ ]:
# Kategorik değişkenleri encode et
label_encoders = {}
categorical_columns = ['genre', 'country', 'directors']

df_processed = df.copy()

for column in categorical_columns:
    label_encoders[column] = LabelEncoder()
    encoded_column_name = f'encoded_{column}'
    df_processed[encoded_column_name] = label_encoders[column].fit_transform(df_processed[column])
    print(f"✅ {column} sütunu encode edildi -> {encoded_column_name}")

print("\n🎯 Encode edilmiş sütunların örnek değerleri:")
encoded_cols = [col for col in df_processed.columns if col.startswith('encoded_')]
df_processed[['title'] + encoded_cols].head()

In [ ]:
# Toplam skor hesapla
score_features = ['avg_vote', 'humor', 'rhythm', 'effort', 'tension', 'erotism']
encoded_features = ['encoded_genre', 'encoded_country', 'encoded_directors']

all_features = score_features + encoded_features
df_processed['toplam_skor'] = df_processed[all_features].sum(axis=1) / len(all_features)

print(f"📊 Toplam skor hesaplandı. Kullanılan özellikler: {all_features}")
print(f"📈 Ortalama toplam skor: {df_processed['toplam_skor'].mean():.2f}")
print(f"📉 Minimum toplam skor: {df_processed['toplam_skor'].min():.2f}")
print(f"📊 Maksimum toplam skor: {df_processed['toplam_skor'].max():.2f}")

In [ ]:
# Sınıflandırma yap (median eşik değeri)
esik_degeri = df_processed['toplam_skor'].median()
df_processed['siniflandirma'] = df_processed['toplam_skor'].apply(
    lambda x: 1 if x >= esik_degeri else 0
)

print(f"🎯 Sınıflandırma tamamlandı. Eşik değeri: {esik_degeri:.2f}")
print(f"⭐ Üst sınıf (1): {sum(df_processed['siniflandirma'] == 1)} film")
print(f"📉 Alt sınıf (0): {sum(df_processed['siniflandirma'] == 0)} film")

# Sınıf dağılımını görselleştir
plt.figure(figsize=(8, 6))
df_processed['siniflandirma'].value_counts().plot(kind='bar', color=['lightcoral', 'lightblue'])
plt.title('Film Sınıflandırma Dağılımı')
plt.xlabel('Sınıf (0: Alt, 1: Üst)')
plt.ylabel('Film Sayısı')
plt.xticks(rotation=0)
plt.show()

## 4. 📊 Keşifsel Veri Analizi (EDA)

In [ ]:
# Film rating dağılımı
plt.figure(figsize=(15, 5))

# avg_vote dağılımı
plt.subplot(1, 3, 1)
plt.hist(df_processed['avg_vote'], bins=15, alpha=0.7, color='skyblue', edgecolor='black')
plt.title('Film Rating Dağılımı')
plt.xlabel('avg_vote')
plt.ylabel('Frekans')

# Film süresi dağılımı
plt.subplot(1, 3, 2)
plt.hist(df_processed['duration'], bins=15, alpha=0.7, color='lightgreen', edgecolor='black')
plt.title('Film Süresi Dağılımı')
plt.xlabel('Süre (dakika)')
plt.ylabel('Frekans')

# Toplam skor dağılımı
plt.subplot(1, 3, 3)
plt.hist(df_processed['toplam_skor'], bins=15, alpha=0.7, color='lightcoral', edgecolor='black')
plt.title('Toplam Skor Dağılımı')
plt.xlabel('Toplam Skor')
plt.ylabel('Frekans')

plt.tight_layout()
plt.show()

In [ ]:
# Duygusal özellikler analizi
emotion_features = ['humor', 'rhythm', 'effort', 'tension', 'erotism']

plt.figure(figsize=(15, 10))

for i, feature in enumerate(emotion_features, 1):
    plt.subplot(2, 3, i)
    plt.hist(df_processed[feature], bins=15, alpha=0.7, edgecolor='black')
    plt.title(f'{feature.capitalize()} Dağılımı')
    plt.xlabel(feature)
    plt.ylabel('Frekans')

plt.tight_layout()
plt.show()

# Duygusal özelliklerin istatistikleri
print("📊 Duygusal Özelliklerin İstatistikleri:")
df_processed[emotion_features].describe().round(2)

## 5. 🔗 Korelasyon Analizi

In [ ]:
# Korelasyon matrisi hesapla
correlation_features = ['avg_vote', 'duration', 'humor', 'rhythm', 'effort', 'tension', 'erotism', 'toplam_skor']
correlation_matrix = df_processed[correlation_features].corr()

# Korelasyon heatmap
plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(
    correlation_matrix,
    mask=mask,
    annot=True,
    cmap='coolwarm',
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": .8},
    fmt='.3f'
)
plt.title('Film Özellikleri Korelasyon Matrisi', fontsize=16, fontweight='bold')
plt.show()

print("\n📋 Korelasyon Matrisi:")
correlation_matrix.round(3)

In [ ]:
# avg_vote ile en yüksek korelasyona sahip özellikler
avg_vote_correlations = correlation_matrix['avg_vote'].drop('avg_vote').sort_values(key=abs, ascending=False)

plt.figure(figsize=(10, 6))
colors = ['red' if x < 0 else 'blue' for x in avg_vote_correlations.values]
bars = plt.barh(range(len(avg_vote_correlations)), avg_vote_correlations.values, color=colors, alpha=0.7)

plt.yticks(range(len(avg_vote_correlations)), avg_vote_correlations.index)
plt.xlabel('Korelasyon Katsayısı')
plt.title('avg_vote ile Diğer Özelliklerin Korelasyonu', fontsize=14, fontweight='bold')
plt.grid(axis='x', alpha=0.3)

# Değerleri bar üzerine yaz
for i, (bar, value) in enumerate(zip(bars, avg_vote_correlations.values)):
    plt.text(value + (0.01 if value >= 0 else -0.01), i, f'{value:.3f}', 
            va='center', ha='left' if value >= 0 else 'right')

plt.tight_layout()
plt.show()

print("\n🎯 avg_vote ile En Yüksek Korelasyonlar:")
for feature, corr in avg_vote_correlations.head(5).items():
    print(f"  {feature}: {corr:.3f}")

## 6. 🎯 Film Öneri Sistemi

In [ ]:
# Öneri sistemi için özellik hazırlama
# avg_vote'u kategorilere ayır
df_processed['selected_avg_vote'] = pd.cut(df_processed['avg_vote'], 
                                         bins=[0, 4, 6, 8, 10], 
                                         labels=['düşük', 'orta', 'iyi', 'mükemmel'])

# Encoded sütunları string'e çevir
for col in ['encoded_genre', 'encoded_country', 'encoded_directors']:
    df_processed[f'selected_{col}'] = df_processed[col].astype(str)

# Duygusal özellikleri kategorilere ayır
emotion_cols = ['humor', 'rhythm', 'effort', 'tension', 'erotism']
for col in emotion_cols:
    df_processed[f'selected_{col}'] = pd.cut(df_processed[col], 
                                           bins=[0, 3, 6, 10], 
                                           labels=['düşük', 'orta', 'yüksek'])

print("✅ Öneri sistemi için özellikler hazırlandı")

# Seçilmiş özellikleri göster
selected_features = [col for col in df_processed.columns if col.startswith('selected_')]
print(f"🎯 Kullanılacak özellikler: {selected_features}")

In [ ]:
# CountVectorizer ile feature matrix oluştur
feature_df = df_processed[selected_features].copy()
for col in feature_df.columns:
    if feature_df[col].dtype.name == 'category':
        feature_df[col] = feature_df[col].astype(str).fillna('unknown')
    else:
        feature_df[col] = feature_df[col].fillna('unknown')

feature_text = feature_df.astype(str).agg(' '.join, axis=1)

vectorizer = CountVectorizer()
feature_matrix = vectorizer.fit_transform(feature_text)

print(f"📊 Feature matrix oluşturuldu: {feature_matrix.shape}")
print(f"🔤 Vocabular boyutu: {len(vectorizer.vocabulary_)}")

In [ ]:
# Cosine similarity hesapla
cosine_sim = cosine_similarity(feature_matrix, feature_matrix)
print(f"📊 Cosine similarity matrisi: {cosine_sim.shape}")
print(f"📈 Ortalama benzerlik skoru: {cosine_sim.mean():.3f}")
print(f"📊 Maksimum benzerlik skoru: {cosine_sim.max():.3f}")

In [ ]:
# Öneri fonksiyonu
def get_movie_recommendations(movie_title, num_recommendations=5):
    """
    Belirli bir film için benzer filmleri önerir
    """
    # Film indeksini bul
    try:
        movie_index = df_processed[df_processed['title'].str.contains(movie_title, case=False, na=False)].index[0]
    except IndexError:
        print(f"'{movie_title}' filmi bulunamadı!")
        return []
    
    # Benzerlik skorlarını al
    similarities = cosine_sim[movie_index]
    
    # En benzer filmlerin indekslerini al
    similar_movies_indices = np.argsort(similarities)[::-1][1:num_recommendations+1]
    
    # Önerilen filmleri return et
    recommendations = []
    for idx in similar_movies_indices:
        movie_info = {
            'title': df_processed.iloc[idx]['title'],
            'similarity_score': similarities[idx],
            'avg_vote': df_processed.iloc[idx]['avg_vote'],
            'genre': df_processed.iloc[idx]['genre'],
            'country': df_processed.iloc[idx]['country']
        }
        recommendations.append(movie_info)
    
    return recommendations

# Test et
test_movie = "Eşkıya"
recommendations = get_movie_recommendations(test_movie, 5)

print(f"\n🎬 '{test_movie}' filmine benzer filmler:")
print("=" * 50)
for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec['title']}")
    print(f"   Benzerlik: {rec['similarity_score']:.3f}")
    print(f"   Rating: {rec['avg_vote']}")
    print(f"   Tür: {rec['genre']}")
    print(f"   Ülke: {rec['country']}\n")

## 7. 📈 Regresyon Analizi

In [ ]:
# Regresyon için özellik seçimi
regression_features = ['humor', 'rhythm', 'effort', 'tension', 'erotism', 
                      'encoded_genre', 'encoded_country', 'encoded_directors', 'duration']
target = 'avg_vote'

X = df_processed[regression_features]
y = df_processed[target]

print(f"📊 Regresyon analizi")
print(f"🎯 Hedef değişken: {target}")
print(f"📋 Özellik sayısı: {len(regression_features)}")
print(f"📊 Örneklem boyutu: {len(X)}")

# Veriyi böl
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\n🔄 Veri bölünmesi:")
print(f"  Eğitim: {len(X_train)} örnek")
print(f"  Test: {len(X_test)} örnek")

In [ ]:
# Lineer regresyon modeli
model = LinearRegression()
model.fit(X_train, y_train)

# Tahminler
y_pred = model.predict(X_test)

# Performans metrikleri
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"📊 Model Performansı:")
print(f"  MSE: {mse:.4f}")
print(f"  R² Score: {r2:.4f}")
print(f"  RMSE: {np.sqrt(mse):.4f}")

# Özellik katsayıları
feature_importance = pd.DataFrame({
    'feature': regression_features,
    'coefficient': model.coef_,
    'abs_coefficient': np.abs(model.coef_)
}).sort_values('abs_coefficient', ascending=False)

print(f"\n🎯 Özellik Önemleri:")
feature_importance

In [ ]:
# Gerçek vs Tahmin grafikleri
plt.figure(figsize=(15, 5))

# Scatter plot: Gerçek vs Tahmin
plt.subplot(1, 3, 1)
plt.scatter(y_test, y_pred, alpha=0.7, color='blue')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Gerçek avg_vote')
plt.ylabel('Tahmin edilen avg_vote')
plt.title('Gerçek vs Tahmin')
plt.grid(alpha=0.3)

# Residual plot
plt.subplot(1, 3, 2)
residuals = y_test - y_pred
plt.scatter(y_pred, residuals, alpha=0.7, color='green')
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Tahmin edilen avg_vote')
plt.ylabel('Artık (Residual)')
plt.title('Artık Grafik')
plt.grid(alpha=0.3)

# Özellik önemleri
plt.subplot(1, 3, 3)
colors = ['red' if x < 0 else 'blue' for x in feature_importance['coefficient']]
plt.barh(range(len(feature_importance)), feature_importance['coefficient'], color=colors, alpha=0.7)
plt.yticks(range(len(feature_importance)), feature_importance['feature'])
plt.xlabel('Katsayı')
plt.title('Özellik Katsayıları')
plt.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## 8. 📋 Sonuçlar ve Değerlendirme

In [ ]:
# Genel rapor
print("🎬 FİLM ÖNERİ SİSTEMİ - SONUÇ RAPORU")
print("=" * 50)

print(f"\n📊 Veri Seti Özeti:")
print(f"  • Toplam film sayısı: {len(df)}")
print(f"  • Ortalama rating: {df['avg_vote'].mean():.2f}")
print(f"  • Rating aralığı: {df['avg_vote'].min():.1f} - {df['avg_vote'].max():.1f}")

print(f"\n🎯 Öneri Sistemi:")
print(f"  • Cosine similarity kullanıldı")
print(f"  • {len(selected_features)} özellik kullanıldı")
print(f"  • Ortalama benzerlik skoru: {cosine_sim.mean():.3f}")

print(f"\n📈 Regresyon Modeli:")
print(f"  • MSE: {mse:.4f}")
print(f"  • R² Score: {r2:.4f}")
print(f"  • En önemli özellik: {feature_importance.iloc[0]['feature']}")

print(f"\n🔗 Korelasyon Analizi:")
print(f"  • En yüksek korelasyon: {avg_vote_correlations.iloc[0]:.3f} ({avg_vote_correlations.index[0]})")
print(f"  • Genel korelasyon seviyesi: Zayıf")

print(f"\n💡 Öneriler:")
print(f"  • Model performansı düşük (R² = {r2:.3f})")
print(f"  • Daha fazla özellik ve veri gerekli")
print(f"  • TF-IDF, Word2Vec gibi gelişmiş yöntemler denenebilir")
print(f"  • Collaborative filtering yaklaşımı eklenebilir")

## 🎯 İnteraktif Demo

In [ ]:
# İnteraktif film önerisi demo
def interactive_movie_demo():
    print("🎬 İNTERAKTİF FİLM ÖNERİSİ")
    print("=" * 30)
    
    print("\nMevcut filmlerden bazıları:")
    sample_movies = df_processed['title'].head(10).tolist()
    for i, movie in enumerate(sample_movies, 1):
        print(f"{i:2d}. {movie}")
    
    # Rastgele bir film seç ve öner
    random_movie = np.random.choice(df_processed['title'])
    print(f"\n🎲 Rastgele seçilen film: {random_movie}")
    
    recommendations = get_movie_recommendations(random_movie, 3)
    print(f"\n✨ '{random_movie}' için öneriler:")
    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. {rec['title']} (Benzerlik: {rec['similarity_score']:.3f})")

# Demo'yu çalıştır
interactive_movie_demo()

## 📝 Notlar ve İyileştirme Önerileri

### ✅ Tamamlanan Görevler:
- Veri ön işleme ve temizleme
- Kategorik verilerin encode edilmesi
- İçerik tabanlı öneri sistemi
- Regresyon analizi
- Korelasyon analizi ve görselleştirme

### 🎯 İyileştirme Önerileri:
1. **Daha Büyük Veri Seti**: Daha fazla film ve özellik eklenebilir
2. **Gelişmiş NLP**: TF-IDF, Word2Vec veya BERT kullanılabilir
3. **Collaborative Filtering**: Kullanıcı-film etkileşimi verileri eklenebilir
4. **Deep Learning**: Neural Collaborative Filtering denenebilir
5. **A/B Testing**: Farklı öneri algoritmalarının karşılaştırılması

### 📊 Model Performansı:
- **Öneri Sistemi**: Çalışıyor ancak daha fazla özellik gerekli
- **Regresyon**: Düşük R² skoru (iyileştirme gerekli)
- **Korelasyon**: Zayıf ilişkiler (ek özellikler eklenebilir)